In [1]:
import os
import json
from PIL import Image
import cv2

In [2]:
class_mapping = {
    "car": 0,
    "motorbike": 1,
    "threewheel": 2,
    "bus": 3,
    "van": 4,
    "truck": 5
}

In [3]:
# Paths
train_json = "dataset/train/ann" # annotations from dataset
train_img = "dataset/train/images" # images
train_output = "dataset/train/labels" # labels for yolo

valid_json = "dataset/valid/ann"
valid_img = "dataset/valid/images"
valid_output = "dataset/valid/labels"

In [4]:
# Check file extensions
img_extensions = set([os.path.splitext(f)[1] for f in os.listdir(train_img)])
ann_extensions = set([os.path.splitext(f)[1] for f in os.listdir(train_json)])

print(f"File extensions in {train_img}: {img_extensions}")
print(f"File extensions in {train_json}: {ann_extensions}")

img_extensions = set([os.path.splitext(f)[1] for f in os.listdir(valid_img)])
ann_extensions = set([os.path.splitext(f)[1] for f in os.listdir(valid_json)])

print(f"File extensions in {valid_img}: {img_extensions}")
print(f"File extensions in {valid_json}: {ann_extensions}")

File extensions in dataset/train/images: {'', '.jpeg', '.png', '.jpg'}
File extensions in dataset/train/ann: {'.json'}
File extensions in dataset/valid/images: {'.jpeg', '.png', '.jpg'}
File extensions in dataset/valid/ann: {'.json'}


In [ ]:
# Path to the preprocessed data (Train)
train_img_preprocessed = "dataset/train/preprocessed"
os.makedirs(train_img_preprocessed, exist_ok=True)

for img_file in os.listdir(train_img):
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(train_img, img_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Gaussian Blur
        blurred = cv2.GaussianBlur(image, (5, 5), 0)

        # Canny Edge
        edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

        # Save the processed image
        output_path = os.path.join(train_img_preprocessed, img_file)
        cv2.imwrite(output_path, edges)
        print(f"Edge detected image saved: {output_path}")

Edge detected image saved: dataset/train/preprocessed\00043_GMC Savana Van 2012.jpg
Edge detected image saved: dataset/train/preprocessed\00077.jpg
Edge detected image saved: dataset/train/preprocessed\00116.jpg
Edge detected image saved: dataset/train/preprocessed\00175_Ford E-Series Wagon Van 2012.jpg
Edge detected image saved: dataset/train/preprocessed\00241.jpg
Edge detected image saved: dataset/train/preprocessed\00294.jpg
Edge detected image saved: dataset/train/preprocessed\00300_Nissan NV Passenger Van 2012.jpg
Edge detected image saved: dataset/train/preprocessed\00391_Mercedes-Benz Sprinter Van 2012.jpg
Edge detected image saved: dataset/train/preprocessed\00394.jpg
Edge detected image saved: dataset/train/preprocessed\00441.jpg
Edge detected image saved: dataset/train/preprocessed\00543.jpg
Edge detected image saved: dataset/train/preprocessed\00613_Nissan NV Passenger Van 2012.jpg
Edge detected image saved: dataset/train/preprocessed\00688.jpg
Edge detected image saved: da

In [ ]:
# Path to the preprocessed data (Valid)
valid_img_preprocessed = "dataset/valid/preprocessed"
os.makedirs(valid_img_preprocessed, exist_ok=True)

for img_file in os.listdir(valid_img):
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(valid_img, img_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Gaussian Blur
        blurred = cv2.GaussianBlur(image, (5, 5), 0)

        # Canny Edge
        edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

        # Save the processed image
        output_path = os.path.join(valid_img_preprocessed, img_file)
        cv2.imwrite(output_path, edges)
        print(f"Edge detected image saved: {output_path}")

Edge detected image saved: dataset/valid/preprocessed\00110.jpg
Edge detected image saved: dataset/valid/preprocessed\00262.jpg
Edge detected image saved: dataset/valid/preprocessed\00790.jpg
Edge detected image saved: dataset/valid/preprocessed\00826.jpg
Edge detected image saved: dataset/valid/preprocessed\01000_GMC Savana Van 2012.jpg
Edge detected image saved: dataset/valid/preprocessed\01444.jpg
Edge detected image saved: dataset/valid/preprocessed\01455.jpg
Edge detected image saved: dataset/valid/preprocessed\01890.jpg
Edge detected image saved: dataset/valid/preprocessed\02136.jpg
Edge detected image saved: dataset/valid/preprocessed\02321.jpg
Edge detected image saved: dataset/valid/preprocessed\02358_Nissan NV Passenger Van 2012.jpg
Edge detected image saved: dataset/valid/preprocessed\02381.jpg
Edge detected image saved: dataset/valid/preprocessed\02504.jpg
Edge detected image saved: dataset/valid/preprocessed\02564_Chevrolet Express Van 2007.jpg
Edge detected image saved: d

In [ ]:
def convert_to_yolo(json_dir, img_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for json_file in os.listdir(json_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(json_dir, json_file)
            base_name = os.path.splitext(json_file)[0]  # Get base name without the extension

            # Try matching with different extensions
            img_filename = None
            for ext in ['.jpg', '.jpeg', '.png']:
                possible_img_path = os.path.join(img_dir, base_name)
                if os.path.exists(possible_img_path):
                    img_filename = base_name
                    break

            # If no matching image is found, skip this JSON file
            if not img_filename:
                print(f"No matching image found for {json_file}")
                continue

            img_path = os.path.join(img_dir, img_filename)

            # Open the image to get dimensions
            try:
                img = Image.open(img_path)
                img_width, img_height = img.size
            except Exception as e:
                print(f"Error opening image {img_path}: {e}")
                continue

            # Open the JSON file
            with open(json_path, 'r') as f:
                data = json.load(f)

            # Prepare YOLO annotation lines
            yolo_annotations = []
            for obj in data.get('objects', []):
                class_name = obj['classTitle']
                if class_name not in class_mapping:
                    print(f"Class '{class_name}' not in class mapping. Skipping.")
                    continue

                class_id = class_mapping[class_name]
                points = obj['points']['exterior']
                x_min, y_min = points[0]
                x_max, y_max = points[1]

                # Calculate YOLO format
                x_center = ((x_min + x_max) / 2) / img_width
                y_center = ((y_min + y_max) / 2) / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height

                # Append to annotations
                yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # Write to output .txt file
            output_txt_path = os.path.join(output_dir, base_name + '.txt')
            with open(output_txt_path, 'w') as out_file:
                out_file.write("\n".join(yolo_annotations))
                print(f"Annotations written for {img_filename} -> {output_txt_path}")

In [8]:
# Create YOLO-compatible annotations for train and validation sets
convert_to_yolo(train_json, train_img_preprocessed, train_output)
convert_to_yolo(valid_json, valid_img_preprocessed, valid_output)

Annotations written for 00043_GMC Savana Van 2012.jpg -> dataset/train/labels\00043_GMC Savana Van 2012.jpg.txt
Annotations written for 00077.jpg -> dataset/train/labels\00077.jpg.txt
Annotations written for 00116.jpg -> dataset/train/labels\00116.jpg.txt
Annotations written for 00175_Ford E-Series Wagon Van 2012.jpg -> dataset/train/labels\00175_Ford E-Series Wagon Van 2012.jpg.txt
Annotations written for 00241.jpg -> dataset/train/labels\00241.jpg.txt
Annotations written for 00294.jpg -> dataset/train/labels\00294.jpg.txt
Annotations written for 00300_Nissan NV Passenger Van 2012.jpg -> dataset/train/labels\00300_Nissan NV Passenger Van 2012.jpg.txt
Annotations written for 00391_Mercedes-Benz Sprinter Van 2012.jpg -> dataset/train/labels\00391_Mercedes-Benz Sprinter Van 2012.jpg.txt
Annotations written for 00394.jpg -> dataset/train/labels\00394.jpg.txt
Annotations written for 00441.jpg -> dataset/train/labels\00441.jpg.txt
Annotations written for 00543.jpg -> dataset/train/labels\00

In [14]:
# Count images and labels in train and valid directories
train_images = len([f for f in os.listdir(train_img_preprocessed) if f.endswith(('.jpg', '.jpeg', '.png'))])
train_labels = len([f for f in os.listdir(train_output) if f.endswith('.txt')])

valid_images = len([f for f in os.listdir(valid_img_preprocessed) if f.endswith(('.jpg', '.jpeg', '.png'))])
valid_labels = len([f for f in os.listdir(valid_output) if f.endswith('.txt')])

# Print results
print(f"Train images: {train_images}, Train labels: {train_labels}")
print(f"Validation images: {valid_images}, Validation labels: {valid_labels}")

Train images: 2097, Train labels: 2097
Validation images: 900, Validation labels: 900


In [ ]:
# Make sure labels are in .txt

labels_dir = "D:/University/Comvis/AOL/withYOLO/dataset/train/labels"

# Iterate through all files in the labels directory
for filename in os.listdir(labels_dir):
    if filename.endswith((".jpg.txt", ".jpeg.txt", ".png.txt")):
        old_path = os.path.join(labels_dir, filename)
        # Replace the extensions
        new_filename = filename.replace(".jpg.txt", ".txt").replace(".jpeg.txt", ".txt").replace(".png.txt", ".txt")
        new_path = os.path.join(labels_dir, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

Renamed: 00043_GMC Savana Van 2012.jpg.txt -> 00043_GMC Savana Van 2012.txt
Renamed: 00077.jpg.txt -> 00077.txt
Renamed: 00116.jpg.txt -> 00116.txt
Renamed: 00175_Ford E-Series Wagon Van 2012.jpg.txt -> 00175_Ford E-Series Wagon Van 2012.txt
Renamed: 00241.jpg.txt -> 00241.txt
Renamed: 00294.jpg.txt -> 00294.txt
Renamed: 00300_Nissan NV Passenger Van 2012.jpg.txt -> 00300_Nissan NV Passenger Van 2012.txt
Renamed: 00391_Mercedes-Benz Sprinter Van 2012.jpg.txt -> 00391_Mercedes-Benz Sprinter Van 2012.txt
Renamed: 00394.jpg.txt -> 00394.txt
Renamed: 00441.jpg.txt -> 00441.txt
Renamed: 00543.jpg.txt -> 00543.txt
Renamed: 00613_Nissan NV Passenger Van 2012.jpg.txt -> 00613_Nissan NV Passenger Van 2012.txt
Renamed: 00688.jpg.txt -> 00688.txt
Renamed: 00711_Nissan NV Passenger Van 2012.jpg.txt -> 00711_Nissan NV Passenger Van 2012.txt
Renamed: 00806.jpg.txt -> 00806.txt
Renamed: 00964_Ford E-Series Wagon Van 2012.jpg.txt -> 00964_Ford E-Series Wagon Van 2012.txt
Renamed: 00972_Nissan NV Passe

In [ ]:
# Make sure labels are in .txt

labels_dir = "D:/University/Comvis/AOL/withYOLO/dataset/valid/labels"

# Iterate through all files in the labels directory
for filename in os.listdir(labels_dir):
    if filename.endswith((".jpg.txt", ".jpeg.txt", ".png.txt")):
        old_path = os.path.join(labels_dir, filename)
        # Replace the extensions
        new_filename = filename.replace(".jpg.txt", ".txt").replace(".jpeg.txt", ".txt").replace(".png.txt", ".txt")
        new_path = os.path.join(labels_dir, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

Renamed: 00110.jpg.txt -> 00110.txt
Renamed: 00262.jpg.txt -> 00262.txt
Renamed: 00790.jpg.txt -> 00790.txt
Renamed: 00826.jpg.txt -> 00826.txt
Renamed: 01000_GMC Savana Van 2012.jpg.txt -> 01000_GMC Savana Van 2012.txt
Renamed: 01444.jpg.txt -> 01444.txt
Renamed: 01455.jpg.txt -> 01455.txt
Renamed: 01890.jpg.txt -> 01890.txt
Renamed: 02136.jpg.txt -> 02136.txt
Renamed: 02321.jpg.txt -> 02321.txt
Renamed: 02358_Nissan NV Passenger Van 2012.jpg.txt -> 02358_Nissan NV Passenger Van 2012.txt
Renamed: 02381.jpg.txt -> 02381.txt
Renamed: 02504.jpg.txt -> 02504.txt
Renamed: 02564_Chevrolet Express Van 2007.jpg.txt -> 02564_Chevrolet Express Van 2007.txt
Renamed: 02743.jpg.txt -> 02743.txt
Renamed: 02878_GMC Savana Van 2012.jpg.txt -> 02878_GMC Savana Van 2012.txt
Renamed: 03137_Nissan NV Passenger Van 2012.jpg.txt -> 03137_Nissan NV Passenger Van 2012.txt
Renamed: 03206.jpg.txt -> 03206.txt
Renamed: 03218_Mercedes-Benz Sprinter Van 2012.jpg.txt -> 03218_Mercedes-Benz Sprinter Van 2012.txt
Re

In [ ]:
# Check if images and labels match

img_dir = "D:/University/Comvis/AOL/withYOLO/dataset/train/preprocessed"
labels_dir = "D:/University/Comvis/AOL/withYOLO/dataset/train/labels"

img_files = {os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))}
label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}

unmatched_labels = label_files - img_files
unmatched_images = img_files - label_files

if unmatched_labels:
    print("Unmatched label files:")
    for label in unmatched_labels:
        print(f"{label}.txt")
if unmatched_images:
    print("Unmatched image files:")
    for img in unmatched_images:
        print(f"{img}.jpg")

if not unmatched_labels and not unmatched_images:
    print("All files are properly matched!")

All files are properly matched!


In [17]:
# Remove unmatched label files
for label in unmatched_labels:
    label_path = os.path.join(labels_dir, f"{label}.txt")
    if os.path.exists(label_path):
        os.remove(label_path)
        print(f"Removed unmatched label file: {label_path}")

# Remove unmatched image files
for img in unmatched_images:
    for ext in ['.jpg', '.jpeg', '.png']:
        img_path = os.path.join(img_dir, f"{img}{ext}")
        if os.path.exists(img_path):
            os.remove(img_path)
            print(f"Removed unmatched image file: {img_path}")

In [18]:
labels_dir = "D:/University/Comvis/AOL/train/labels"

for label_file in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_file)
    with open(label_path, "r") as f:
        lines = f.readlines()

    corrected_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            print(f"Malformed annotation in {label_file}: {line.strip()}")
            continue

        class_id, x_center, y_center, width, height = map(float, parts)
        # Check if coordinates are out of bounds
        if 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1:
            corrected_lines.append(line)
        else:
            print(f"Invalid coordinates in {label_file}: {line.strip()}")

    # Overwrite the label file with corrected annotations
    with open(label_path, "w") as f:
        f.writelines(corrected_lines)

In [ ]:
# Check if there are still images with no labels and labels with non images

img_dir = "D:/University/Comvis/AOL/withYOLO/dataset/train/preprocessed"
labels_dir = "D:/University/Comvis/AOL/withYOLO/dataset/train/labels"

img_files = {os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))}
label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}

missing_labels = img_files - label_files
missing_images = label_files - img_files

print(f"Images without labels: {missing_labels}")
print(f"Labels without images: {missing_images}")

# Check if YOLO can access a sample label file
sample_label = os.path.join(labels_dir, list(label_files)[0] + ".txt")
if os.path.exists(sample_label):
    with open(sample_label, 'r') as f:
        print(f"Sample label content:\n{f.read()}")
else:
    print("Sample label file not found.")


Images without labels: set()
Labels without images: set()
Sample label content:
5 0.493721 0.514675 0.943485 0.849057


# Training

Run the block below to set up yolov5

In [ ]:
# import subprocess

# # Clone the YOLOv5 repository
# subprocess.run(["git", "clone", "https://github.com/ultralytics/yolov5.git"])

# # Change directory to yolov5
# import os
# os.chdir("yolov5")

# # Install requirements
# subprocess.run(["pip", "install", "-r", "requirements.txt"])

Setup data.yaml first which includes:
train -> train images
valid -> valid images
nc -> number of clusters
names -> names of clusters

More info on YOLO:
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data

Run in Terminal to start training:

cd yolov5

python train.py --img 640 --batch 16 --epochs 10 --data data.yaml --weights yolov5s.pt

In [ ]:
# yolo5_dir = "D:/University/Comvis/AOL/withYOLO/yolov5"
# os.chdir(yolo5_dir)

# subprocess.run([
#     "python", "train.py",
#     "--img", "640",
#     "--batch", "16",
#     "--epochs", "10",
#     "--data", "data.yaml",
#     "--weights", "yolov5s.pt"
# ])

CompletedProcess(args=['python', 'train.py', '--img', '640', '--batch', '16', '--epochs', '10', '--data', 'data.yaml', '--weights', 'yolov5s.pt'], returncode=1)

For additional epochs (5 more epochs):

python train.py --img 640 --batch 16 --epochs 5 --data data.yaml --weights runs/train/expX/weights/last.pt

# Evaluation

Run validation dataset:

python val.py --weights runs/train/expX/weights/best.pt --data data.yaml --img 640

# Application

Setting up flask:

pip install flask opencv-python torch torchvision ultralytics

To run the app:
1. Open app.py
2. Run the file from VSC
3. It will open website in local

using "python app.py" on the terminal somehow doesn't work